In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from glob import glob
from pathlib import Path
import pandas as pd 
from pkg import *

datasets = [1,2,3]
myPaths = path.PathManager(datasets=datasets)
myProcessor = process.Processor(paths=myPaths, datasets=datasets)
params = myProcessor.get_parameters()
print(params)
f.get_counts(paths=myPaths, datasets=datasets)

myDatasetManager = data.DatasetManager(paths=myPaths, datasets=datasets)
train_loader, test_loader = f.prepare(data_manager=myDatasetManager, batch_size=10)

Water backgrounds initialized: 3
{'01': {'clen': 0.15, 'photon_energy': 6000}, '02': {'clen': 0.15, 'photon_energy': 7000}, '03': {'clen': 0.15, 'photon_energy': 8000}}
Paths refreshed for dataset ['01', '02', '03'].
Directory: peaks/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks/03
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/03
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/03
	Total files: 45
	Normal images: 30
	Empty images: 15
Actual percentage of empty images: 33.33333333333333% across peaks, wate

In [2]:
import torch; print('PyTorch version:', torch.__version__)
import torchvision; print('Torchvision version:', torchvision.__version__)
from torchvision.ops import nms; print('NMS loaded successfully')
import torch; print(torch.__version__); print(torch.cuda.is_available())

PyTorch version: 2.1.2.post103
Torchvision version: 0.16.1
NMS loaded successfully
2.1.2.post103
False


In [3]:
b = background.BackgroundSubtraction(threshold=20)
type(myPaths.total_paths.overlays)

all_data = b.main(inputs=myPaths.total_paths.overlays)
print(all_data)
# all_data.to_csv('all_data.csv')

Batch Overlay Files Data:
          coordinate  radius  average_intensity  center_pixel_intensity  \
0        (182, 1661)       1           5.250000                    21.0   
1        (182, 1661)       2           5.958333                    21.0   
2        (182, 1661)       3           6.187500                    21.0   
3        (182, 1661)       4           6.162500                    21.0   
4        (219, 1166)       1           7.125000                    24.0   
...              ...     ...                ...                     ...   
317315  (2104, 1013)       4           6.787500                    22.0   
317316  (2109, 1074)       1           5.875000                    21.0   
317317  (2109, 1074)       2           6.916667                    21.0   
317318  (2109, 1074)       3           6.270833                    21.0   
317319  (2109, 1074)       4           6.587500                    21.0   

        peak_intensity_estimate  
0                     15.750000  
1    

In [4]:
f.check_attributes(paths=myPaths, datasets=f.convert2str(datasets), dir_type='peak')
print(myProcessor.get_parameters())

f.retrieve_attributes(file_path=myPaths.total_paths.water_background[0])
f.check_attributes(paths=myPaths, datasets=f.convert2str(datasets), dir_type='label')

/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00019.h5 conforms to expected attributes.
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00019.h5:
Expected Peak: True, Actual Peak: True
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00014.h5 conforms to expected attributes.
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00014.h5:
Expected Peak: False, Actual Peak: False
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00004.h5 conforms to expected attributes.
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00004.h5:
Expected Peak: False, Actual Peak: False
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00009.h5 conforms to expected attributes.
/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_

True

In [5]:
f.get_counts(paths=myPaths, datasets=datasets)
# %run process_directory.py ../../images  --percent_empty 0.5 
# f.get_counts(paths=myPaths, datasets=datasets)

Paths refreshed for dataset ['01', '02', '03'].
Directory: peaks/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/01
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/02
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks/03
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: labels/03
	Total files: 45
	Normal images: 30
	Empty images: 15
Directory: peaks_water_overlay/03
	Total files: 45
	Normal images: 30
	Empty images: 15


In [6]:
model1 = m.BasicCNN1()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = {
    'num_epochs': 2,
    'num_classes': 2,
    'batch_size': 10,
    'threshold':0.5,
    'loader': [train_loader,test_loader],
    'train_size': len(train_loader.dataset),
    'test_size': len(test_loader.dataset),
    'criterion': criterion,
    'optimizer': optimizer,
    'device': device,
    'model': model1,
    }

# self, model: nn.Module, loader: list, criterion, optimizer, device, cfg: dict, feature: str, labels: list
t = train_eval.TrainTestModels(model=model1, loader=[train_loader,test_loader], criterion=criterion, optimizer=optimizer, device=device, cfg=cfg, feature='peak', labels=['water','background'])
t.epoch_loop()

TypeError: TrainTestModels.__init__() got an unexpected keyword argument 'model'

In [ ]:
test_peaks = myPaths.get_peak_image_paths(dataset=dataset)[0]
test_water = myPaths.get_peaks_water_overlay_image_paths(dataset=dataset)[0]
test_overlay = myPaths.get_peaks_water_overlay_image_paths(dataset=dataset)[0]
param1, param2 = (0.5, 9000), (0.5, 9000, True)
f.assign_attributes(file_path=test_peaks, params=param1)

Attributes ['params'] assigned to /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00020.h5
